In [253]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [254]:
athletes = pd.read_csv('data/summerOly_athletes.csv')
hosts = pd.read_csv('data/summerOly_hosts.csv')
medal_counts = pd.read_csv('data/summerOly_medal_counts.csv', encoding='utf-8')
medal_counts = medal_counts.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
programs = pd.read_csv('data/summerOly_programs.csv', encoding='utf-8')
full_medal = pd.read_csv('data/full_medal_data.csv')

In [5]:
year_to_country_code = {
    1896: "GRE",  # Athens, Greece
    1900: "FRA",  # Paris, France
    1904: "USA",  # St. Louis, USA
    1908: "GBR",  # London, United Kingdom
    1912: "SWE",  # Stockholm, Sweden
    1916: None,   # Cancelled
    1920: "BEL",  # Antwerp, Belgium
    1924: "FRA",  # Paris, France
    1928: "NED",  # Amsterdam, Netherlands
    1932: "USA",  # Los Angeles, USA
    1936: "GER",  # Berlin, Germany
    1940: None,   # Cancelled
    1944: None,   # Cancelled
    1948: "GBR",  # London, United Kingdom
    1952: "FIN",  # Helsinki, Finland
    1956: "AUS",  # Melbourne, Australia
    1960: "ITA",  # Rome, Italy
    1964: "JPN",  # Tokyo, Japan
    1968: "MEX",  # Mexico City, Mexico
    1972: "GER",  # Munich, Germany
    1976: "CAN",  # Montreal, Canada
    1980: "URS",  # Moscow, Soviet Union
    1984: "USA",  # Los Angeles, USA
    1988: "KOR",  # Seoul, South Korea
    1992: "ESP",  # Barcelona, Spain
    1996: "USA",  # Atlanta, USA
    2000: "AUS",  # Sydney, Australia
    2004: "GRE",  # Athens, Greece
    2008: "CHN",  # Beijing, China
    2012: "GBR",  # London, United Kingdom
    2016: "BRA",  # Rio de Janeiro, Brazil
    2020: "JPN",  # Tokyo, Japan
    2024: "FRA",  # Paris, France
    2028: "USA",  # Los Angeles, USA
    2032: "AUS",  # Brisbane, Australia
}

In [6]:
hosts['NOC'] = hosts['Year'].map(year_to_country_code)

In [257]:
NOC = medal_counts['NOC'].str.replace('\xa0','').unique()

In [258]:
medal_counts_LR = medal_counts

medal_counts_LR = medal_counts_LR.sort_values(by=['NOC', 'Year']).reset_index(drop=True)

medal_counts_LR['Predicted_Medals'] = np.nan

for index, row in medal_counts_LR.iterrows():
    country = row['NOC']
    year = row['Year']
    #print(f"Processing: Country={country}, Year={year}")
    
    # Filter previous years' data for the same country
    historical_data = medal_counts_LR[(medal_counts_LR['NOC'] == country) & (medal_counts_LR['Year'] < year)]
    #print(historical_data)
    
    # Check if there is enough data to train a model
    if len(historical_data) >= 2:  # At least 2 data points needed for Linear Regression
        # Prepare training data
        X_train = historical_data[['Year']].values
        y_train = historical_data['Total'].values
        
        # Train the model
        model = LinearRegression()
        model.fit(X_train, y_train)
        
        # Predict the medal count for the current year
        X_test = np.array([[year]])  # Current year as input
        predicted_medals = model.predict(X_test)[0]
        
        # Store the prediction in the DataFrame
        medal_counts_LR.at[index, 'Predicted_Medals'] = predicted_medals

In [255]:
# Display the updated DataFrame
medal_counts_LR.iloc[1:10]

,Rank,NOC,Gold,Silver,Bronze,Total,Year,Predicted_Medals
1,79,Afghanistan,0,0,1,1,2012,NaN
2,80,Albania,0,0,2,2,2024,NaN
3,42,Algeria,0,0,2,2,1984,NaN
4,34,Algeria,1,0,1,2,1992,NaN
5,34,Algeria,2,0,1,3,1996,2.000000
6,42,Algeria,1,1,3,5,2000,3.000000
7,68,Algeria,0,1,1,2,2008,5.571429
8,50,Algeria,1,0,0,1,2012,3.400000
9,63,Algeria,0,2,0,2,2016,2.110000


In [263]:
full_medal.iloc[1:10]

,Rank,NOC,Gold,Silver,Bronze,Total,Year,Country Code,Is_Host,Ex-Host,Host Continent,Team Size,Participated Events,Total Events
1,2,Greece,10,18,19,47,1896,GRE,1,1,1,148.0,39,43.0
2,3,Germany,6,5,2,13,1896,GER,0,0,1,94.0,27,43.0
3,4,France,5,4,2,11,1896,FRA,0,0,1,26.0,18,43.0
4,5,Great Britain,2,3,2,7,1896,GBR,0,0,0,25.0,19,43.0
5,6,Hungary,2,1,3,6,1896,HUN,0,0,1,18.0,14,43.0
6,7,Austria,2,1,2,5,1896,AUT,0,0,1,8.0,8,43.0
7,8,Australia,2,0,0,2,1896,AUS,0,0,0,5.0,5,43.0
8,9,Denmark,1,2,3,6,1896,DEN,0,0,1,15.0,12,43.0
9,10,Switzerland,1,2,0,3,1896,SUI,0,0,0,8.0,5,43.0


In [280]:
medal_counts_LR = full_medal.sort_values(by=['NOC', 'Year']).reset_index(drop=True)

medal_counts_LR['Predicted Medals'] = np.nan
medal_counts_LR['Predicted Proportion of Medals'] = np.nan

for index, row in medal_counts_LR.iterrows():
    country = row['NOC']
    year = row['Year']
    #print(f"Processing: Country={country}, Year={year}")
    
    # Filter previous years' data for the same country
    historical_data = medal_counts_LR[(medal_counts_LR['NOC'] == country) & (medal_counts_LR['Year'] < year)]
    #print(historical_data)
    
    # Check if there is enough data to train a model
    if len(historical_data) >= 2:  # At least 2 data points needed for Linear Regression
        # Prepare training data
        X_train = historical_data[['Year']].values
        y_train = historical_data['Total'].values
        
        # Train the model
        model = LinearRegression()
        model.fit(X_train, y_train)
        
        # Predict the medal count for the current year
        X_test = np.array([[year]])  # Current year as input
        predicted_medals = model.predict(X_test)[0]
        
        # Store the prediction in the DataFrame
        medal_counts_LR.at[index, 'Predicted Medals'] = predicted_medals
        medal_counts_LR.at[index, 'Predicted Proportion of Medals'] = predicted_medals/(3* row['Total Events'])

In [281]:
# Display the updated DataFrame
medal_counts_LR.iloc[1:1000]

,Rank,NOC,Gold,Silver,Bronze,Total,Year,Country Code,Is_Host,Ex-Host,Host Continent,Team Size,Participated Events,Total Events,Predicted Medals,Predicted Proportion of Medals
1,0,ARU,0,0,0,0,1988,ARU,0,0,0,12.0,10,237.0,NaN,NaN
2,0,ARU,0,0,0,0,1992,ARU,0,0,0,5.0,4,257.0,NaN,NaN
3,0,ARU,0,0,0,0,1996,ARU,0,0,1,3.0,3,271.0,0.000000,0.000000
4,0,ARU,0,0,0,0,2000,ARU,0,0,0,5.0,5,300.0,0.000000,0.000000
5,0,ARU,0,0,0,0,2004,ARU,0,0,0,4.0,4,301.0,0.000000,0.000000
6,0,ARU,0,0,0,0,2008,ARU,0,0,0,2.0,2,302.0,0.000000,0.000000
7,0,ARU,0,0,0,0,2012,ARU,0,0,0,4.0,4,302.0,0.000000,0.000000
8,0,ARU,0,0,0,0,2016,ARU,0,0,0,7.0,6,306.0,0.000000,0.000000
9,0,ARU,0,0,0,0,2020,ARU,0,0,0,4.0,4,339.0,0.000000,0.000000
10,0,ARU,0,0,0,0,2024,ARU,0,0,0,7.0,7,329.0,0.000000,0.000000
